In [2]:
import pandas as pd
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow import lite
from ml_service.settings import BASE_DIR
#import joblib


file_path = os.path.join(BASE_DIR.parent, 'data_input.csv')

# Load data_input.csv file
df = pd.read_csv(file_path)

print(file_path)

# Load data_input.csv file
df = pd.read_csv(file_path)


def train_decision_tree(diabetic_level_range):
    # Filter data for the specified diabetic level range
    df_filtered = df[df['diabetic'].between(*diabetic_level_range)]

    # Assuming 'diabetic_level' is the target variable
    X_text = (df_filtered['meal'].fillna('') + ' ' + df_filtered['exercise'].fillna('')).astype(str)
    y = df_filtered['diabetic']

    # Use CountVectorizer for text vectorization
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(X_text)

    # Create and train the Decision Tree Classifier
    classifier = DecisionTreeClassifier()
    classifier.fit(X, y)

    return classifier, vectorizer

def get_recommendations(diabetic_level, diabetic_level_range):
    # Check if the provided diabetic level is within the specified range
    if diabetic_level not in range(*diabetic_level_range):
        if diabetic_level > 200:
            return "Your diabetic level is very high. Please consult with a doctor."
        elif diabetic_level < 70:
            return "You are not a diabetic person. Chill!"
        else:
            return "Invalid diabetic level. Please provide a level within the specified range."

    # Filter the rows based on the provided diabetic level
    recommendations = df[df['diabetic'] == diabetic_level]

    if recommendations.empty:
        # Train the model using the specified diabetic level range
        classifier, vectorizer = train_decision_tree(diabetic_level_range)

        # Create a sample input for prediction (modify as needed)
        sample_input_text = 'sample_meal sample_exercise'
        sample_input_vectorized = vectorizer.transform([sample_input_text])

        # Make a prediction for the user's diabetic level
        predicted_diabetic_level = classifier.predict(sample_input_vectorized)[0]

        # Filter the recommendations based on the predicted diabetic level
        recommendations = df[df['diabetic'] == predicted_diabetic_level]

    if recommendations.empty:
        return "No recommendations found for the specified diabetic level."

    # Extract meal and exercise recommendations
    meal_recommendations = recommendations['meal'].values[0].split(', ')
    exercise_recommendations = recommendations['exercise'].values[0].split(', ')

    # Format the result into JSON
    formatted_result = {
        'meal': format_recommendations(meal_recommendations),
        'exercise': format_recommendations(exercise_recommendations)
    }

    return formatted_result

def format_recommendations(recommendations):
    formatted_dict = {'morning': [], 'afternoon': [], 'evening': []}

    for recommendation in recommendations:
        parts = recommendation.split(': ')
        if len(parts) == 2:
            time, name = parts
            formatted_dict[time.lower()].append({'name': name.strip()})

    return formatted_dict


# Specify the diabetic level range for training
training_diabetic_level_range = (70, 201)


# Test for diabetic level 70
# i want to pass the input (user_diabetic_level) from the android user
user_diabetic_level = 110
result = get_recommendations(user_diabetic_level, training_diabetic_level_range)
result

ModuleNotFoundError: No module named 'ml_service'